# Direction

- Please submit your Jupyter Notebook for the Take-home Exam to the Brightspace with the name below:

  - `danl-m1-exam-LASTNAME-FIRSTNAME.ipynb`\
  ( e.g., `danl-m1-exam-choe-byeonghak.ipynb` )

- The due is March 26, 2024, 7:00 P.M.

- Please send Byeong-Hak an email (`bchoe@geneseo.edu`) if you have any questions.

-   Please prepare a Jupyter/Python Notebook (`*.ipynb`) to address all questions.

-   Make at least some simple comment (`# ...`) in each question.

-   Make one text cell to explain things in each question.

-   Import Python libraries you need here.

In [12]:
import pandas as pd
from itables import init_notebook_mode
from itables import show

from google.colab import data_table
data_table.enable_dataframe_formatter()

<br><br><br>



# Part 1 - Beer Markets

<br>

<p align="center">
  <img src="https://bcdanl.github.io/lec_figs/beer.png" width="600px">
</p>

<br>


Load DataFrame for Part 1:

In [4]:
beer_mkts = pd.read_csv('https://bcdanl.github.io/data/beer_markets.csv')
show(beer_mkts)

- Each observation in `beer_mkts` is a household-level transaction record for a purchase of beer.

## Variable Description


<br>

- `hh`: an identifier of the household;
- `X_purchase_desc`: details on the purchased item;
- `quantity`: the number of items purchased;
- `brand`: Bud Light, Busch Light, Coors Light, Miller Lite, or Natural Light;
- `dollar_spent`: total dollar value of purchase;
- `beer_floz`: total volume of beer, in fluid ounces;
- `price_per_floz`: price per fl.oz. (i.e., beer spent/beer floz);
- `container`: the type of container;
- `promo`: Whether the item was promoted (coupon or otherwise);
- `market`: Scan-track market (or state if rural);
- demographic data, including gender, marital status, household income, class of work, race, education, age, the size of household, and whether or not the household has a microwave or a dishwasher.


<br>

<br>

## Question 1

- Find the top 5 beer markets in terms of the number of households that purchased beer.
- Find the top 5 beer markets in terms of the amount of total beer consumption.

*Answer*:

In [25]:
# the top 5 beer markets in terms of the number of households that purchased beer
q1a = (
    beer_mkts
    .drop_duplicates(subset = ['hh', 'market'])
    .groupby('market')
    .size()
    .reset_index(name = 'n')
    .nlargest(5, 'n', keep = 'all')
)

show(q1a)

In [26]:
q1b = (
    beer_mkts
    .groupby('market')[['beer_floz']]
    .sum()
    .nlargest(5, 'beer_floz', keep = "all")
)

show(q1b)

<br>

## Question 2
- For households that purchased *BUD LIGHT* at least once, what fraction of households did purchase only *BUD LIGHT*?

- For households that purchased *BUSCH LIGHT* at least once, what fraction of households did purchase only *BUSCH LIGHT*?

- For households that purchased *COORS LIGHT* at least once, what fraction of households did purchase only *COORS LIGHT*?

- For households that purchased *MILLER LITE* at least once, what fraction of households did purchase only *MILLER LITE*?

- For households that purchased *NATURAL LIGHT* at least once, what fraction of households did purchase only *NATURAL LIGHT*?

- Which beer brand does have the largest proportion of such loyal consumers?


*Answer*:

Below calculates the number of households that have purchased BUD LIGHT at least once:



In [17]:
hh_bud = (
    beer_mkts
    .query('brand == "BUD LIGHT"')['hh']
    .unique()
)

# number of households that have purchased BUD LIGHT at least once
len(hh_bud)

4657

Below calculates the number of households that have purchased only BUD LIGHT:



In [18]:
n_hh_loyal_bud = (
    beer_mkts[ beer_mkts['hh'].isin(hh_bud) ][['hh','brand']]
    .value_counts()
    .reset_index(name = 'n_purchases')
    .groupby('hh')
    .size()
    .reset_index(name = 'n_brands')
    .query('n_brands == 1')
)

# number of households that only bought `BUD`
len(n_hh_loyal_bud)

3074

Therefore, among households that have purchased BUD LIGHT at least once, the proportion only bought BUD is:



In [19]:
len(n_hh_loyal_bud) / len(hh_bud)


0.6600815975950183

Since we should repeat this over brands, let’s define a custom function and use for-loop:



In [22]:
def loyal_consumer_pct(df, item):
  hh_item = (
    df[ df['brand'] == item ]['hh']
    .unique()
    )

  n_hh_tot = len(hh_item)

  n_hh_loyal_item = (
    df[ df['hh'].isin(hh_item) ][['hh','brand']]
    .value_counts()
    .reset_index(name = 'n_purchases')
    .groupby('hh')
    .size()
    .reset_index(name = 'n')
    .query('n == 1')
    )

  n_hh_loyal = len(n_hh_loyal_item)

  return 100 * n_hh_loyal / n_hh_tot

for i in beer_mkts['brand'].unique():
  print(i, loyal_consumer_pct(beer_mkts, i))

BUD LIGHT 66.00815975950182
BUSCH LIGHT 47.2972972972973
COORS LIGHT 63.9080459770115
MILLER LITE 63.129890453834115
NATURAL LIGHT 50.96234309623431


Therefore, BUD LIGHT’s consumers tend to be most loyal.

Note that more than 50% of households stick to only one beer brand they prefer!



<br><br><br>




# Part 2 - Search Engine Marketing


<br>

<p align="center">
  <img src="https://bcdanl.github.io/lec_figs/ebay-google-mkt.png" width="800px">
</p>

<br>


Load DataFrame for Part 2:

In [23]:
paidsearch = pd.read_csv('https://bcdanl.github.io/data/paidsearch.csv')
show(paidsearch)

##  Variable Description

- `dma`: an identification number of a designated market (DM) area `i`, defined by Nielsen (e.g., Boston, Los Angeles. [This webpage](https://www.spstechnical.com/DMACodes.htm) provides the list of DMA names.)
- `treatment_period`: 0 if date is before May 22, 2012 and 1 after.
- `search_stays_on`: 1 if the paid-search goes off in dma `i`, 0 otherwise.
- `revenue`: eBay's sales revenue for dma `i` and date `t`

<br>

## Question 3

Summarize the mean value of `revenue` for each group of `search_stays_on` and for each `date`.

*Answer*:

In [27]:
q3 = (
    paidsearch
    .groupby(['date', 'search_stays_on'])
    .agg(revenue_mean = ('revenue', 'mean'))
    .reset_index()
)

show(q3)

<br>

## Question 4

Calculate the log difference between mean revenues in each group of `search_stays_on`. (This is the log of the average revenue in group of `search_stays_on` == 1 minus the log of the average revenue in group of `search_stays_on` == 0.)

- For example, consider the following two observations:

In [ ]:
# date        the daily mean vale of `revenue`   search_stays_on
# 1-Apr-12    93650.68                           0
# 1-Apr-12    120277.57                          1

- The log difference of daily mean revenues between the two group of `search_stays_on` for date 1-Apr-12 is log(120277.57) - log(93650.68).

- *Hint*: Below example shows how we can calculate the log of `Value` using `np.log()`:

In [ ]:
import pandas as pd
import numpy as np

# Creating a DataFrame
data = {'Value': [1, 10, 100, 1000, 10000]}
df = pd.DataFrame(data)

# Calculating logarithms
df['Log_natural'] = np.log(df['Value'])

*Answer*:

In [32]:
import numpy as np
q4 = (
    q3
    .pivot(
        index = 'date',
        columns = 'search_stays_on',
        values = 'revenue_mean'
    )
    .rename(columns = {0:'revenue_search_on',
                       1:'revenue_search_off'})
    .assign(
        log_revenue_search_on =
            lambda df: np.log(df['revenue_search_on']) - np.log(df['revenue_search_off'])
    )
)

show(q4)

<br><br>



# Part 3 - NFL

<br>

<p align="center">
  <img src="https://bcdanl.github.io/lec_figs/nfl.png" width="300px">
</p>

<br>

- The following is the DataFrame for Part 3.

In [34]:
NFL2023_stuffs = pd.read_csv('https://bcdanl.github.io/data/NFL2023_stuffs.csv')
show(NFL2023_stuffs)

- `NFL2023_stuffs` is the DataFrame that contains information about NFL games in year 2022, in which the unit of observation is a single play for each drive in a NFL game.


## Variable description

- `play_id`: Numeric play identifier that when used with `game_id` and `drive` provides the unique identifier for a single play
- `game_id`: Ten digit identifier for NFL game.
- `drive`: Numeric drive number in the game.
- `week`: Season week.
- `posteam`: String abbreviation for the team with possession.
- `qtr`: Quarter of the game (5 is overtime).
- `half_seconds_remaining`: Numeric seconds remaining in the half.
- `down`: The down for the given play.
  - Basically you get four attempts (aka downs) to move the ball 10 yards (by either running with it or passing it).
  - If you make 10 yards then you get another set of four downs.
- `pass`: Binary indicator if the play was a pass play.
- `wp`: Estimated winning probability for the `posteam` given the current situation at the start of the given play.

<br>

## Question 5
In DataFrame, `NFL2023_stuffs`, remove observations for which values of `posteam` is missing.

*Answer*:

In [35]:
q5 = (
    NFL2023_stuffs
    .dropna(subset = ['posteam'])
)

q5['posteam'].notna().sum() == q5.shape[0]

True

<br>


## Question 6
- Calculate the mean value of `pass` for each `posteam` when all the following conditions hold:
  1. `wp` is greater than 20% and less than 75%;
  2. `down` is less than or equal to 2; and
  3. `half_seconds_remaining` is greater than 120.

*Answer*:

In [36]:
q6 = (
    NFL2023_stuffs
    .query('wp > .2 & wp < .75 & down <= 2 & half_seconds_remaining > 120')
    .groupby('posteam')[['pass']]
    .agg(
        pass_mean = ('pass', 'mean')
    ).
    sort_values('pass_mean', ascending = False)
)

show(q6)

<br>


## Question 7
- Consider the following DataFrame, `NFL2023_epa`:

In [37]:
NFL2023_epa = pd.read_csv('https://bcdanl.github.io/data/NFL2023_epa.csv')
show(NFL2023_epa)

### Variable Description for `NFL2023_epa`

  - `play_id`: Numeric play identifier that when used with `game_id` and `drive` provides the unique identifier for a single play
  - `game_id`: Ten digit identifier for NFL game.
  - `drive`: Numeric drive number in the game.
  - `posteam`: String abbreviation for the team with possession.
  - `passer`: Name of the player who passed a ball to a receiver by initially taking a three-step drop and backpedaling into the pocket to make a pass. (Mostly, they are quarterbacks)
  - `receiver`: Name of the receiver.
  - `epa`: Expected points added (EPA) by the `posteam` for the given play.

<br>

- Create the DataFrame, `NFL2023_stuffs_EPA`, that includes
  1. All the variables in the DataFrame, `NFL2023_stuffs`;
  2. The variables, `passer`, `receiver`, and `epa`, from the DataFrame, `NFL2023_epa`.
by joining the two DataFrames.

- In the resulting DataFrame, `NFL2023_stuffs_EPA`, remove observations with `NA` in `passer`.

*Answer*:

In [39]:
NFL2023_stuffs_EPA = (
    NFL2023_stuffs
    .merge(NFL2023_epa, on = ['play_id', 'game_id', 'drive'], how = 'left')
    .dropna(subset = ['passer'])
)

show(NFL2023_stuffs_EPA)

<br>